# Setting up the GPU

In [ ]:
import tensorflow as tf

# Check for GPU and set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPUs are available and memory growth is set")
    except RuntimeError as e:
        print(e)

# Loading the ECG data for all subjects

In [ ]:
import os
import pandas as pd

# List to store individual subject DataFrames
data_frames = []

# Iterate over the subject numbers
for subject_number in range(2, 18):
    # Construct the file path for each subject
    file_path = f'/kaggle/input/wesad-dataset/S{subject_number}_respiban.txt'
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Read the file into a DataFrame
        df = pd.read_csv(file_path, delimiter='\t', skiprows=3, header=None)
        # Select the first three columns
        df_subset = df.iloc[:, 2]
        # Append the DataFrame to the list
        print(df_subset.shape)
 
        df_subset.columns = [f'Subject_{subject_number}']
        
        data_frames.append(df_subset)
    else:
        print(f'File not found for subject {subject_number}')

# Concatenate all DataFrames into a single DataFrame
data_1 = pd.concat(data_frames,axis=1)

# Print the shape of the combined data
print(data_1.shape)

# The ECG signal required to have some modification to converting it from raw signal values to SI unit.

In [ ]:
import numpy as np
# Define the constants
chan_bit = 2 ** 16
vcc = 3
# Apply the equation to the dataset
data_wesad = data_1.applymap(lambda x: ((x / chan_bit - 0.5) * vcc) if not np.isnan(x) else np.nan)
# Print the updated dataset
print(data_wesad)

# Sample plotting of the ECG signal

In [ ]:
import matplotlib.pyplot as plt

# Select the data for the first subject
subject_data = data_wesad.iloc[:550, 4]  # Assuming the first column represents the first subject

# Create a time axis for the plot
time_axis = range(550)

# Plot the data
plt.plot(time_axis, subject_data)
plt.xlabel('Time')
plt.ylabel('Voltage(mV)')
plt.title('Plot of First 1000 Data Points - Subject 1')
plt.show()

# Loading the start and end of every session(stress/baseline) for each subjects. The file WESAD_mins.xlsx was made by the author(based on the WESAD documentation) for the simplification of the process.

In [ ]:
import pandas as pd

# Specify the path to your .xlsx file
file_path = '/kaggle/input/wesad-dataset/WESAD_mins.xlsx'

# Read the .xlsx file into a DataFrame
df = pd.read_excel(file_path)
df_sub = df.iloc[:, 1:]
# Print the DataFrame
print(df_sub)

# Setting up the data points according to the baseline and stress time

In [ ]:
num_intervals_base = 119
num_intervals_tsst = 59

base_interval = (df_sub['Base_end'] - df_sub['Base_start']) / (num_intervals_base + 1)
tsst_interval = (df_sub['Tsst_end'] - df_sub['TSST_Start']) / (num_intervals_tsst + 1)

# Create the new dataset
new_df = pd.DataFrame()
new_df['base_start'] = df_sub['Base_start']
for i in range(1, num_intervals_base + 1):
    new_df[f'base_s{i}'] = df_sub['Base_start'] + i * base_interval
new_df['base_end'] = df_sub['Base_end']
new_df['tsst_start'] = df_sub['TSST_Start']
for i in range(1, num_intervals_tsst + 1):
    new_df[f'tsst_s{i}'] = df_sub['TSST_Start'] + i * tsst_interval
new_df['tsst_end'] = df_sub['Tsst_end']
df_lebel=(new_df*700*60).T
# Print the new dataset
print(df_lebel.shape)

# Observing the absolute lowest session segment data points in the dataset.

In [ ]:
min_diffs = df_lebel.diff(axis=0).abs().min()

# Find the absolute lowest value among the minimum differences
absolute_lowest = np.floor(min_diffs.min()).astype(int)

# Print the absolute lowest value
print(absolute_lowest)

# Slicing every subjects data of same shape to avoid the mismatch in the data point. Also setting up the label.

In [ ]:
import pandas as pd
import numpy as np

samples = []
lebels = []
# Iterate over the subjects
for subject_number in range(15):
    # Read the first dataset for the current subject
    df1 = data_wesad.iloc[:, subject_number] # Replace with your own file path
    df2 = df_lebel.iloc[:,subject_number]
    #print(subject_number)
    indices = df2.values.astype(int).flatten()
    # Iterate over the indices
    for i in range(len(indices) - 1):
        if i != 20:  # Exclude the 5th sample
            start = indices[i]
            end = start+absolute_lowest
            sample = df1.iloc[start-1:end].values  # Cut the sample from the first dataset
            sample = pd.Series(sample)
            samples.append(sample)
            #print(sample.shape)
    lebel=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
           0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
           0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
           0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
           0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
           0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
           1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
           1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
           1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    lebels = lebels+lebel

# Concatenate the samples along a new axis
concatenated = pd.concat(samples,axis=1).T
lebel_all = pd.DataFrame(lebels)

# Print the shape of the concatenated dataset
print(concatenated.shape)
print(lebel_all.shape)

# Downsamplin the dataset from (700 Hz to 256 Hz to reduce the computationl cost and the improvement of the performance

In [ ]:
import numpy as np
from scipy.signal import resample
original_dataset = concatenated.T

# Assuming original_dataset has shape (450, 40608)
downsampled_dataset = np.zeros((2700, 256 * (6768 // 700)))  # Downsampling to 256 Hz
print(original_dataset[0].shape)
for i in range(original_dataset.shape[1]):
    original_signal = original_dataset[i]
    downsampled_signal = resample(original_signal, 256 * (6768 // 700))
    downsampled_dataset[i, :] = downsampled_signal
wesad_x=downsampled_dataset.T
print(wesad_x.shape)  # Output: (90, 58418)

# Observing the downsampled dataset.

In [ ]:
import matplotlib.pyplot as plt

# Select the data for the first subject
wesad_x = pd.DataFrame(wesad_x)
subject_data = wesad_x.iloc[:200, 4]  # Assuming the first column represents the first subject

# Create a time axis for the plot
time_axis = range(200)

# Plot the data
plt.plot(time_axis, subject_data)
plt.xlabel('Time')
plt.ylabel('Voltage(mV)')
plt.title('Plot of First 1000 Data Points - Subject 1')
plt.show()

# Transposing the dataset.

In [ ]:
X= wesad_x.T
y=lebel_all

# Building the model

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, GlobalMaxPooling1D, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

input_layer = Input(shape=(2304, 1))

# Shared CNN layers
def shared_cnn(input_layer, filters1, kernel_size1, filters2, kernel_size2, filters3, kernel_size3):
    conv1 = Conv1D(filters=filters1, kernel_size=kernel_size1, activation='relu')(input_layer)
    maxpool1 = MaxPooling1D(pool_size=2)(conv1)
    batch_norm1 = BatchNormalization()(maxpool1)

    conv2 = Conv1D(filters=filters2, kernel_size=kernel_size2, activation='relu')(batch_norm1)
    maxpool2 = MaxPooling1D(pool_size=2)(conv2)
    batch_norm2 = BatchNormalization()(maxpool2)

    conv3 = Conv1D(filters=filters3, kernel_size=kernel_size3, activation='relu')(batch_norm2)
    maxpool3 = MaxPooling1D(pool_size=2)(conv3)
    batch_norm3 = BatchNormalization()(maxpool3)

    return batch_norm3

# Shared BiLSTM layer with attention
def shared_bilstm_with_attention(shared_cnn_output, units, return_sequences, num_heads):
    bilstm_output = Bidirectional(LSTM(units=units, return_sequences=return_sequences))(shared_cnn_output)
    attention_heads = []
    for _ in range(num_heads):
        attention_head = Attention()([bilstm_output, bilstm_output])
        attention_heads.append(attention_head)

    # Concatenate the outputs of attention heads
    multi_attention = Concatenate(axis=-1)(attention_heads)
    global_pool = GlobalMaxPooling1D()(multi_attention)
    return global_pool

# Apply shared CNN and BiLSTM with attention to each input path with different parameters
shared_cnn_output1 = shared_cnn(input_layer, filters1=64, kernel_size1=10, filters2=128, kernel_size2=5, filters3=256, kernel_size3=3)
shared_bilstm_output1 = shared_bilstm_with_attention(shared_cnn_output1, units=64, return_sequences=True, num_heads=4)

shared_cnn_output2 = shared_cnn(input_layer, filters1=32, kernel_size1=14, filters2=64, kernel_size2=8, filters3=128, kernel_size3=5)
shared_bilstm_output2 = shared_bilstm_with_attention(shared_cnn_output2, units=32, return_sequences=True, num_heads=2)

# Concatenate the outputs of both paths
concatenated_output = Concatenate(axis=-1)([shared_bilstm_output1, shared_bilstm_output2])

# Dense layers
dense1 = Dense(units=256, activation='relu', kernel_regularizer=l2(0.01))(concatenated_output)
batch_norm4 = BatchNormalization()(dense1)
drop1 = Dropout(0.4)(batch_norm4)

dense2 = Dense(units=128, activation='relu', kernel_regularizer=l2(0.01))(drop1)
batch_norm5 = BatchNormalization()(dense2)
drop2 = Dropout(0.4)(batch_norm5)

# Output layer
output = Dense(units=1, activation='sigmoid')(drop2)

# Model
model = Model(inputs=input_layer, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# from here the LOOCV special code starts. 

In [ ]:
features= wesad_x.T
print(features.shape)
labels=lebel_all
print(labels.shape)

# Here, LOOCV test-train splitting (only 1 subject is considered as validation data. the rests are as train data.) is done. Subjects =[1 to 17] sub = 1 means subject 1 of the Vollmer dataset will be considered as test data, while other subjects will be considered as train data.

In [ ]:

def generate_train_test_data(test_number, test_interval=180):
    if test_number <= 0:
        raise ValueError("Test number should be greater than 0.")
    #if test_number * test_interval > len(features):
        #raise ValueError("Test number exceeds the total number of datasets.")

    start_idx = (test_number - 1) * test_interval
    end_idx = start_idx + test_interval

 
    test_indices = list(range(start_idx, end_idx))
    print(test_indices)
    train_indices = list(range(len(features)))
    train_indices = [i for i in train_indices if i not in test_indices]

    #print(test_slice)
    #print(train_indices)

    X_test = features.iloc[test_indices,:]
    #print(X_test.shape)
    y_test = labels.iloc[test_indices]
    
    X_train = features.iloc[train_indices,:]
    y_train = labels.iloc[train_indices,:]

    return X_train, X_test, y_train, y_test

# Example usage for getting train and test data for the first 6 datasets  5  9
X_train, X_test, y_train, y_test = generate_train_test_data(test_number=13)

# Now, perform the standard scaling as before
from sklearn.preprocessing import StandardScaler

scaling = StandardScaler()
X_train = scaling.fit_transform(X_train)
X_test = scaling.transform(X_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
y_train_flatten = y_train.values.flatten()

X_majority = X_train[y_train_flatten == 0, :]
X_minority = X_train[y_train_flatten == 1, :]
print(X_minority.shape)
print(X_majority.shape)

# train data augmentation

In [ ]:
X_minority_augmented = np.zeros((2*X_minority.shape[0], 2304))
print(X_minority_augmented.shape)
j=0
i=0
#print(int(X_minority.shape[0]/2))
for i in range(int(X_minority.shape[0]/2)):
    i = i*2
    #print(i)
    x1=X_minority[i,:]
    x2=X_minority[i+1,:]
    x_combined = np.concatenate((X_minority[i,:], X_minority[i+1,:]))
    #print(x_combined.shape)
    x3 = x_combined[768:3072]
    x4 = x_combined[1792:4096]
    #print(x3.shape)
    #print(x2.shape)
        
    X_minority_augmented[j] = x1  #np.roll(x1, 2000)
    j=j+1    
    X_minority_augmented[j] = x2  #np.roll(x1, 2000)
    j=j+1
    X_minority_augmented[j] = x3  #np.roll(x1, 2000)
    j=j+1
    #x1=X_minority[i,:]
    X_minority_augmented[j] = x4 #np.roll(x1, 2000)x1
    j=j+1
    #i=i+1
    #print(i)
#print(i)
print(X_minority_augmented.shape)

In [ ]:
X_train_augmented = np.vstack((X_majority, X_minority_augmented))
y_train_augmented = np.hstack((np.zeros(X_majority.shape[0]), np.ones(X_minority_augmented.shape[0])))
# shuffle the data

idx = np.random.permutation(X_train_augmented.shape[0])
#print(idx)
X_train_augmented = X_train_augmented[idx]
y_train_augmented = pd.DataFrame(y_train_augmented[idx])
print(X_train_augmented.shape)
#print(y_train_augmented[0])
X_train=X_train_augmented
y_train=y_train_augmented

# Training of the model

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
# Define early stopping and checkpoint callbacks
es = EarlyStopping(monitor='val_loss', mode='min', patience=100)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# Train the model with early stopping and checkpoint callbacks
history = model.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_test, y_test), callbacks=[es, mc])

# calculating the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from keras.models import load_model
# Load the saved model
model = load_model('best_model.h5')

# Predict the class probabilities for the test set
y_pred = model.predict(X_test)

# Convert the probabilities into class labels using a threshold of 0.5
y_pred_classes = (y_pred > 0.5).astype(int)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)
print(cm)

# loss vs epoch graph

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

# Print loss vs. epoch curve
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# auc score calculation

In [ ]:
# Calculate predictions for the test set
y_pred = model.predict(X_test)

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_pred)

# Print AUC score
print('AUC Score:', auc_score)

# ROC curve plotting

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# Calculate predictions for the test set
y_pred = model.predict(X_test)

# Calculate false positive rate, true positive rate, and thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_pred)

# Plot ROC curve
plt.plot(fpr, tpr, label='ROC Curve (AUC = {:.2f})'.format(auc_score))
plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend()
plt.show()